# COMP5046 Assignment 1
*Make sure you change the file name with your unikey.*

# Readme
*If there is something to be noted for the user, please mention here.* 

*If you are planning to implement a program with Object Oriented Programming style*

***Visualising the comparison of different results is a good way to justify your decision.***

# 1 - Data Preprocessing (Personality chat datasets)

## 1.1. Download Dataset (Personality chat datasets)

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Please comment your code
import json
import re

import tensorflow as tf
import numpy as np
import pandas as pd
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
id_comic = '12c7PwE2BKsrCIBx_Ve3qGkn_toAJ0IMg'
downloaded = drive.CreateFile({'id':id_comic}) 
downloaded.GetContentFile('qna_chitchat_the_comic.tsv')
#https://drive.google.com/file/d/12c7PwE2BKsrCIBx_Ve3qGkn_toAJ0IMg/view?usp=sharing

In [0]:
id_friend = '11Eee3u_yfy4MT5UFTts-p86gocMXiOco'
downloaded = drive.CreateFile({'id':id_friend}) 
downloaded.GetContentFile('qna_chitchat_the_friend.tsv')
#https://drive.google.com/file/d/11Eee3u_yfy4MT5UFTts-p86gocMXiOco/view?usp=sharing

id_professional = '12Dn7am7lOZrzCHYsEJgg-xzrQByc-llC'
downloaded = drive.CreateFile({'id':id_professional}) 
downloaded.GetContentFile('qna_chitchat_the_professional.tsv')
#https://drive.google.com/file/d/12Dn7am7lOZrzCHYsEJgg-xzrQByc-llC/view?usp=sharing

In [0]:
from io import open


data_com = open("qna_chitchat_the_comic.tsv", "r", encoding="utf-8")
data_fri = open("qna_chitchat_the_friend.tsv", "r", encoding="utf-8")
data_pro = open("qna_chitchat_the_professional.tsv", "r", encoding="utf-8")

## 1.2. Preprocess data (Personality chat datasets)

*You are required to describe which data preprocessing techniques were conducted with justification of your decision. *

Here are four techniques conducted: punctuations removal, decapitalisation, tokenisation and stopwords removal.

1.  Punctuations removal
     
     Removing punctuations in the text is a cleaning process of the raw data.  Here, 're' is imported to realize the process. As we know that FastText with CBOW will generate great dimensional vectors for each word. Punctuations and non-English characters are not available. In particular, when we fetch a page from a website, a lot labels of html were loaded together. As a result, the cleaning process is necessary.
2.  Decapitalisation

     According to the characteristics of English, decapitalisation is useful to reduce the number of training words. More specifically, two words with the same spelling and semantic but one is uppercase the other is lowercase, they will be treated as different words in FastText. Decapitalisation is used for reducing repitition of words and improving calculating efficiency.
3.   Tokenisation
    
      For questions in the raw data, we need to tokenize each sentence as independent words. But for the answers, we treat each single answer as a token.
4.   Stopwords removal

      Stopwords are unnecessary in a sentence. Removing them usually has no effect on understanding the meaning of a whole sentence. In a text, there will be a large number of function words, pronouns or verbs and nouns without specific meanings. These words are of no help to the text analysis, so we hope to remove these stopwords.





In [0]:
import json
import re
import string
import tensorflow as tf
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
df_com = pd.read_csv('qna_chitchat_the_comic.tsv', sep="\t")
df_fri = pd.read_csv('qna_chitchat_the_friend.tsv', sep="\t")
df_pro = pd.read_csv('qna_chitchat_the_professional.tsv', sep="\t")

# 2 - Model Implementation

## 2.1. Word Embeddings

*You are required to describe which model was implemented (i.e. Word2Vec with CBOW, FastText with SkipGram, etc.) with justification of your decision *

Here I would like to use FastText with CBOW.

Firstly, FastText is an open source word vector computing and text categorization tool developed by facebook. As there are many popular models to learn such representations ignore the morphology of words, by assigning a
distinct vector to each word. This is a limitation, especially for morphologically rich languages with large vocabularies and many rare words. FastText uses a new approach based on the skip-gram model, where each word is represented as a bag of character n-grams. A vector representation is associated to each character n-gram, words being represented as the sum of these representations. This method is fast, allowing to trainmodels on large corpus quickly. (*Piotr Bojanowski∗and Edouard Grave∗and Armand Joulin and Tomas Mikolov from Facebook AI Research, 15 Jul 2016*). In a word, the value of fastText is that it provides a more readable and modular implementation of word2vec.

Besides, in the CBOW method, the surrounding words are used to predict the center words, making use of the prediction result of the center words. The gradient desent method is used to constantly adjust the vector of the surrounding words. When the training is completed, each word will be used as the central word, and the word vectors of the surrounding words will be adjusted, so as to obtain the word vectors of all the words in the whole text. While, Skip-gram uses the central word to predict the words around it. In SkipGram, gradient decent is used to adjust the word vectors of the center words constantly to predict results of the surrounding words. Finally, the word vectors of all words in the text are obtained.

In other words, CBOW is inferring the target word from the original statement; Skip-Gram, on the other hand, it predicts the original sentence from the target word. CBOW is suitable for small databases, while Skip-Gram performs better in large corpus.

As I used Microsoft BotBuilder chat datasets for word embeddings. The dataset is not that large. Besides, There are a large number of high-frequency words in the corpus. Here, CBOW is more efficient because it takes much less predicted time than SkipGram.

### 2.1.1. Download Dataset for Word Embeddings



Here I still use Microsoft BotBuilder chat datasets. 

The Cornell Movie Dialog Corpus is complex and the amount of its data is large. Moreover, the format of Q&A is not uniform. As I use FastText with CBOW, which is not suitable for very complex data. So Microsoft BotBuilder chat datasets are better choices.

In [0]:
# Please comment your code
datacom=df_com
datafri=df_fri
datapro=df_pro

### 2.1.2. Data Preprocessing for Word Embeddings

The data preprocessing is the same as that in 1.2.

In [0]:
# Please comment your code

def makesentence(data):
  parse_text=[]
  for i in range(data.shape[0]):    
      s1=[data['Question'][i]]
      s2=data['Answer'][i]
      parse_text.append([s1,s2])
      
  return parse_text

In [0]:
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
def make_tokenlist(text): 
  for i in text:
      content_text = re.sub(r'\([^)]*\)', '', i[0][0])

      normalized_text = ""
      for string in content_text:####
          tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
          #normalized_text.append(tokens)
          normalized_text += tokens
      i[0][0]= normalized_text.split()
  new_text=[]
  for i in text:
    new_i=[]
    new_str=""
    for j in i[0][0]:
      new_i.append(j)
    new_i.append(i[1])
    new_text.append(new_i)
  filtered_sentence = [w for w in new_text if not w in stopwords.words()]
  return filtered_sentence

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 2.1.3. Build Word Embeddings Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

size: length of the word vector (defaulted=100)

window: The size of the context window (defaulted=5)

min_count: It is used to discard the words that appear less than min_count times in the training corpus. As we have paddings in datasets, it should be equal to 1.

workers: The number of worker threads to train the model (=faster training with multicore machines).

sg: Training algorithm --- CBOW if sg=0, otherwise skip-gram.


In [0]:
from gensim.models import Word2Vec
from gensim.models import FastText
textcom=makesentence(datacom)
textfri=makesentence(datafri)
textpro=makesentence(datapro)

normalized_textcom=make_tokenlist(textcom)
normalized_textfri=make_tokenlist(textfri)
normalized_textpro=make_tokenlist(textpro)

/usr/local/lib/python3.6/dist-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


### 2.1.4. Train Word Embeddings Model

In [0]:
ft_cbow_model_com=FastText(normalized_textcom, size=100, window=5, min_count=1, workers=4, sg=0)
ft_cbow_model_fri=FastText(normalized_textfri, size=100, window=5, min_count=1, workers=4, sg=0)
ft_cbow_model_pro=FastText(normalized_textpro, size=100, window=5, min_count=1, workers=4, sg=0)

### 2.1.5. Save Word Embeddings Model

In [0]:
ft_cbow_model_com
ft_cbow_model_fri
ft_cbow_model_pro

In [0]:
from gensim.test.utils import common_texts, get_tmpfile
path = get_tmpfile("word2vec.ft_cbow_model_com")
ft_cbow_model_com.save("word2vec.ft_cbow_model_com")
  
uploaded = drive.CreateFile({'title': 'models_comic.ckpt'})
uploaded.SetContentFile('word2vec.ft_cbow_model_com')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1dc-R1bQeWdnQ3VJ7oxqmy_Rtz768MhJH


In [0]:
path = get_tmpfile("word2vec.ft_cbow_model_fri")
ft_cbow_model_fri.save("word2vec.ft_cbow_model_fri")
  
uploaded1 = drive.CreateFile({'title': 'models_friend.ckpt'})
uploaded1.SetContentFile('word2vec.ft_cbow_model_fri')
uploaded1.Upload()
print('Uploaded file with ID {}'.format(uploaded1.get('id')))

Uploaded file with ID 1xRbzt3YklqID-miTMeosJZKIkcRTz3cl


In [0]:
path = get_tmpfile("word2vec.ft_cbow_model_pro")
ft_cbow_model_pro.save("word2vec.ft_cbow_model_pro")
  
uploaded2 = drive.CreateFile({'title': 'models_professional.ckpt'})
uploaded2.SetContentFile('word2vec.ft_cbow_model_pro')
uploaded2.Upload()
print('Uploaded file with ID {}'.format(uploaded2.get('id')))

Uploaded file with ID 1NkRwrD4_5Tt_bPzj6yMTSJIAatvwL9Ti


### 2.1.6. Load Word Embeddings Model

In [0]:
# Please comment your code
#id_comic_model = '1qO8UAV33BX6y-O48Acs_blQbUErOoCsx'
id_comic_model = '1NgJGO9uP1whtvZR1p8LOkL8XSu49_Lew'
downloaded = drive.CreateFile({'id':id_comic_model}) 
downloaded.GetContentFile('models_comic.ckpt')

#id_friend_model = '1oAjwBZhOOM-j27EcUFF-RCzRlJxNdStB'
id_friend_model = '1W3tEtRNsc0HThNgKQh1pRHssEdWNa-KZ'
downloaded1 = drive.CreateFile({'id':id_friend_model}) 
downloaded1.GetContentFile('models_friend.ckpt')

#id_professional_model = '16kkEL6fg3Eg4Nvw4phkyqNSXEaZ43blA'
id_professional_model ='1NkRwrD4_5Tt_bPzj6yMTSJIAatvwL9Ti'
downloaded2 = drive.CreateFile({'id':id_professional_model}) 
downloaded2.GetContentFile('models_professional.ckpt')

## 2.2. Seq2Seq model

### 2.2.1. Apply/Import Word Embedding Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

max_input_words_amount: 

Here, we use max(len(tokenized_q, max_input_words_amount) to get max_input_words_amount, which will be applied to the next N-1 seq2seq model. It is used for getting paddings to let the input length consistent.

num_dic, dic_len: 

They are used for building the targets of the seq2seq model. They are applied to two-dimensional array of targets.

seq_data:

It is an important list for getting word vectors to make batches of the seq2seq model.

unique_words: 

A list for storing nonredundant preprocessed words in order to get paddings.


In [0]:
# Please comment your code

model_com = FastText.load("models_comic.ckpt")
model_fri = FastText.load("models_friend.ckpt")
model_pro = FastText.load("models_professional.ckpt")

### 2.2.2. Build Seq2Seq Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

learning_rate :

The learning rate determines the speed at which the parameter is moved to the optimal value. If the learning rate is too high, it is likely to exceed the optimal value; if the learning rate is too low, the optimization efficiency may be too low and the convergence time will be extremely long. Here we set it as 0.002.
 
 n_hidden :
 
 If the number of hidden neurons is too small, the network can not have the necessary learning ability and information processing ability. On the contrary, too much neurons will not only greatly increase the complexity of the network structure, but also make the network fall into local minima more easily in the learning process, and make the learning speed of the network become very slow. Here n_hidden equals to 128.

 n_class :
 
 It varies according to different models, it equals to dic_len.
 
 n_input :
 
 It equals to 100. (length of a word vector. )
 

In [0]:
# Please comment your code
import re
import string
#import pickle
def build_seq(df,model):
  seq_data = []
  whole_words = []
  max_input_words_amount = 0
  max_output_words_amount = 1

  
  for index, row in df.iterrows():
  
    question = row.Question.lower()
    answer = row.Answer
    
    seq_data.append([question,answer])
    
    # we need to tokenise question 
    for i in question:
        if i in string.punctuation: 
            question = question.replace(i," ")
    for i in question:
        if i in string.digits:
            question = question.replace(i," ")
     
    tokenized_q = question.split()
    #tokenized_q=[w for w in tokenized_qes if not w in stopwords.words()]###
    
    # we do not need to tokenise answer (because we implement N to One model)
    # make a list with only one element (whole sentence)
    tokenized_a =[answer]
    
    ###</You need to fill here>###
    
    
    # add question list and answer list (one element)
    whole_words+=tokenized_q
    whole_words += tokenized_a
    #whole_words.append(tokenized_a)
    
    # we need to decide the maximum size of input word tokens
    max_input_words_amount = max(len(tokenized_q), max_input_words_amount)
    

# we now have a vacabulary list
  unique_words = list(set(whole_words))

# adding special tokens in the vocabulary list    
# _B_: Beginning of Sequence
# _E_: Ending of Sequence
# _P_: Padding of Sequence - for different size input
# _U_: Unknown element of Sequence - for different size input

  unique_words.append('_B_')
  unique_words.append('_E_')
  unique_words.append('_P_')
  unique_words.append('_U_')


  num_dic = {n: i for i, n in enumerate(unique_words)}

  dic_len = len(num_dic)
  
  
  #for key, value in num_dic.items():
     ###d.iteritems: an iterator over the (key, value) items
   # if (key!='_B_')&(key!='_E_')&(key!='_P_')&(key!='_U_'):
    #  vec=model[key]
     # value=vec
  #p_num_dic=pi(num_dic) 
  #p_seq_data=pi(seq_data)
  return num_dic,dic_len,seq_data,max_input_words_amount,unique_words

In [0]:
seq_com=build_seq(datacom,model_com)
seq_fri=build_seq(datafri,model_fri)
seq_pro=build_seq(datapro,model_pro)

In [0]:
com_dic,com_len,com_seq,com_max,com_unique=seq_com

In [0]:
fri_dic,fri_len,fri_seq,fri_max,fri_unique=seq_fri

In [0]:
pro_dic,pro_len,pro_seq,pro_max,pro_unique=seq_pro

In [0]:
def get_target_a(sentence,num_dic):
    tokenized_sentence = [sentence]
    ids = []

    for token in tokenized_sentence:
        
        if token in num_dic:
            ids.append(num_dic[token])
        else:
            ids.append(num_dic['_U_'])
              

    return ids

In [0]:
#import pickle
def get_vectors_q(sentence,model,num_dic,max_):
    
    # tokenise the sentence
    
    max_input_words_amount = max_
    sen = sentence.lower()
    for i in sen:
        if i in string.punctuation: 
            sen = sen.replace(i," ")
    for i in sen:
        if i in string.digits:
            sen = sen.replace(i," ")
            
    #filtered_sentence = [w for w in tokens if not w in stopwords.words()]
    tokens =  sen.split()
    tokenized_sentence = [w for w in tokens if not w in stopwords.words()]
     ###stop words
    
   
    diff = max_input_words_amount - len(tokenized_sentence)
    
    # add paddings if the word is shorter than the maximum number of words    
    for x in range(diff):
        
        tokenized_sentence.append('_?_')
               
    
    data = tokens_to_ids(tokenized_sentence,model,num_dic)
         
    
        
    return data

def get_vectors_a(sentence,model,num_dic):    
    tokenized_sentence = [sentence]
    data = tokens_to_ids(tokenized_sentence,model,num_dic)
    
    return data
    

# convert tokens to index
def tokens_to_ids(tokenized_sentence,model,num_dic):
    ids = []

    for token in tokenized_sentence:
      try:
        #if token in num_dic:
          ids.append(model[token])
      except KeyError as e:
        #else:
          ids.append(np.random.rand(100))

    return ids


# generate a batch data for training/testing
def make_batch(seq_data,model,num_dic,dic,max_):
    input_batch = []
    output_batch = []
    target_batch = []

    for seq in seq_data:  
        
        # Input for encoder cell, convert question to vector
 
        input_data = get_vectors_q(seq[0],model,num_dic,max_) 
      
        # Input for decoder cell, Add '_B_' at the beginning of the sequence data
       
        output_data = [np.zeros(100)]#[np.random.rand(100)]
     
        output_data += get_vectors_a(seq[1],model,num_dic)
        
        # Output of decoder cell (Actual result), Add '_E_' at the end of the sequence data
       
        target = get_target_a(seq[1],dic)
        target.append(dic['_E_'])
        #target = get_vectors_a(seq[1],model,num_dic)
        #target.append(np.array([0] * 100))
        
        array_input=np.array(input_data)
        array_output=np.array(output_data)
        # Convert each token vector to one-hot encode data
        input_batch.append(array_input)
        output_batch.append(array_output)
        target_batch.append(target)
        
       

    return input_batch, output_batch, target_batch

In [0]:
com_key=[]
for key in com_dic:
  com_key.append(key)
  #p_com_key=pi(com_key)
fri_key=[]
for key in fri_dic:
  fri_key.append(key)
  #p_fri_key=pi(fri_key)
pro_key=[]
for key in pro_dic:
  pro_key.append(key)
  #p_pro_key=pi(pro_key)

In [0]:
#tf.reset_default_graph()
def build(com_len):
    learning_rate = 0.002
    n_hidden = 128

    n_class = com_len
    n_input = 100

    ### Neural Network Model
    tf.reset_default_graph()

    # encoder/decoder shape = [batch size, time steps, input size]
    enc_input = tf.placeholder(tf.float32, [None, None, n_input])
    dec_input = tf.placeholder(tf.float32, [None, None, n_input])

    # target shape = [batch size, time steps]
    targets = tf.placeholder(tf.int64, [None, None])


    # Encoder Cell
    with tf.variable_scope('encode'):
        enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
        enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)

        outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,
                                                dtype=tf.float32)
    # Decoder Cell
    with tf.variable_scope('decode'):
        dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
        dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)

        # [IMPORTANT] Setting enc_states as inital_state of decoder cell
        outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input,
                                                  initial_state=enc_states,
                                                  dtype=tf.float32)

    model = tf.layers.dense(outputs, n_class, activation=None)

    cost = tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(
                    logits=model, labels=targets))

    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

 
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    return sess,model,cost,optimizer,enc_input,dec_input,targets
    


### 2.2.3. Train Seq2Seq Model

In [0]:

sess,model,cost,optimizer,enc_input,dec_input,targets=build(com_len)

input_batch, output_batch, target_batch = make_batch(com_seq,model_com,com_key,com_dic,com_max)


Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [0]:
saver_com = tf.train.Saver()
total_epoch = 5000

#input_batch=lo(p_input_batch)
#output_batch=lo(p_output_batch)
#target_batch=lo(p_target_batch)
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={enc_input: input_batch,
                                  dec_input: output_batch,
                                  targets: target_batch})
    if epoch % 100 == 0:
        print('Epoch:', '%04d' % (epoch + 1),
              'cost =', '{:.6f}'.format(loss))

print('Epoch:', '%04d' % (epoch + 1),
      'cost =', '{:.6f}'.format(loss))
print('Training completed')
save_path_com = saver_com.save(sess,"save/model_com.ckpt")

Epoch: 0001 cost = 6.380381
Epoch: 0101 cost = 2.236198
Epoch: 0201 cost = 1.485604
Epoch: 0301 cost = 0.472460
Epoch: 0401 cost = 0.119365
Epoch: 0501 cost = 0.045360
Epoch: 0601 cost = 0.026266
Epoch: 0701 cost = 0.015445
Epoch: 0801 cost = 0.012115
Epoch: 0901 cost = 0.008448
Epoch: 1001 cost = 0.009590
Epoch: 1101 cost = 0.005430
Epoch: 1201 cost = 0.004650
Epoch: 1301 cost = 0.005830
Epoch: 1401 cost = 0.003846
Epoch: 1501 cost = 0.003131
Epoch: 1601 cost = 0.002933
Epoch: 1701 cost = 0.002351
Epoch: 1801 cost = 0.002114
Epoch: 1901 cost = 0.001981
Epoch: 2001 cost = 2.467515
Epoch: 2101 cost = 0.010895
Epoch: 2201 cost = 0.005530
Epoch: 2301 cost = 0.004488
Epoch: 2401 cost = 0.003840
Epoch: 2501 cost = 0.004113
Epoch: 2601 cost = 0.001919
Epoch: 2701 cost = 0.001927
Epoch: 2801 cost = 0.001649
Epoch: 2901 cost = 0.002074
Epoch: 3001 cost = 0.001647
Epoch: 3101 cost = 0.001216
Epoch: 3201 cost = 0.000940
Epoch: 3301 cost = 0.001538
Epoch: 3401 cost = 0.000852
Epoch: 3501 cost = 0

In [0]:
sess,model,cost,optimizer,enc_input,dec_input,targets=build(fri_len)

n_class=fri_len
input_batch2, output_batch2, target_batch2 = make_batch(fri_seq,model_fri,fri_key,fri_dic,fri_max)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [0]:
saver_fri = tf.train.Saver() 
total_epoch1=5000
for epoch in range(total_epoch1):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={enc_input: input_batch2,
                                  dec_input: output_batch2,
                                  targets: target_batch2})
    if epoch % 100 == 0:
        print('Epoch:', '%04d' % (epoch + 1),
              'cost =', '{:.6f}'.format(loss))

print('Epoch:', '%04d' % (epoch + 1),
      'cost =', '{:.6f}'.format(loss))
print('Training completed')
save_path_fri = saver_fri.save(sess,"save/model_fri.ckpt")

Epoch: 0001 cost = 6.414020
Epoch: 0101 cost = 2.251807
Epoch: 0201 cost = 1.471490
Epoch: 0301 cost = 0.465624
Epoch: 0401 cost = 0.109619
Epoch: 0501 cost = 0.046409
Epoch: 0601 cost = 0.026690
Epoch: 0701 cost = 0.018336
Epoch: 0801 cost = 0.012088
Epoch: 0901 cost = 0.009323
Epoch: 1001 cost = 0.005849
Epoch: 1101 cost = 0.005402
Epoch: 1201 cost = 0.005102
Epoch: 1301 cost = 0.004303
Epoch: 1401 cost = 0.003911
Epoch: 1501 cost = 0.003239
Epoch: 1601 cost = 0.002584
Epoch: 1701 cost = 0.472208
Epoch: 1801 cost = 0.033196
Epoch: 1901 cost = 0.017856
Epoch: 2001 cost = 0.010487
Epoch: 2101 cost = 0.008392
Epoch: 2201 cost = 0.006193
Epoch: 2301 cost = 0.005484
Epoch: 2401 cost = 0.003149
Epoch: 2501 cost = 0.003948
Epoch: 2601 cost = 0.003406
Epoch: 2701 cost = 0.002515
Epoch: 2801 cost = 0.002670
Epoch: 2901 cost = 0.002060
Epoch: 3001 cost = 0.002138
Epoch: 3101 cost = 0.001803
Epoch: 3201 cost = 0.001496
Epoch: 3301 cost = 0.001495
Epoch: 3401 cost = 0.001756
Epoch: 3501 cost = 0

In [0]:
sess,model,cost,optimizer,enc_input,dec_input,targets=build(pro_len)

n_class=pro_len
input_batch3, output_batch3, target_batch3 = make_batch(pro_seq,model_pro,pro_key,pro_dic,pro_max)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [0]:
total_epoch2=5000
saver_pro = tf.train.Saver()
for epoch in range(total_epoch2):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={enc_input: input_batch3,
                                  dec_input: output_batch3,
                                  targets: target_batch3})
    if epoch % 100 == 0:
        print('Epoch:', '%04d' % (epoch + 1),
              'cost =', '{:.6f}'.format(loss))

print('Epoch:', '%04d' % (epoch + 1),
      'cost =', '{:.6f}'.format(loss))
print('Training completed')
save_path_pro = saver_pro.save(sess,"save/model_pro.ckpt")

Epoch: 0001 cost = 6.474985
Epoch: 0101 cost = 2.236373
Epoch: 0201 cost = 1.463908
Epoch: 0301 cost = 0.383279
Epoch: 0401 cost = 0.095638
Epoch: 0501 cost = 0.045677
Epoch: 0601 cost = 0.027191
Epoch: 0701 cost = 0.017265
Epoch: 0801 cost = 0.013254
Epoch: 0901 cost = 0.010115
Epoch: 1001 cost = 0.008563
Epoch: 1101 cost = 0.006029
Epoch: 1201 cost = 0.004776
Epoch: 1301 cost = 0.003710
Epoch: 1401 cost = 0.003550
Epoch: 1501 cost = 0.003348
Epoch: 1601 cost = 0.042030
Epoch: 1701 cost = 0.010983
Epoch: 1801 cost = 0.008197
Epoch: 1901 cost = 0.005574
Epoch: 2001 cost = 0.004556
Epoch: 2101 cost = 0.003580
Epoch: 2201 cost = 0.003237
Epoch: 2301 cost = 0.002678
Epoch: 2401 cost = 0.002307
Epoch: 2501 cost = 0.002137
Epoch: 2601 cost = 0.001857
Epoch: 2701 cost = 0.002017
Epoch: 2801 cost = 0.001482
Epoch: 2901 cost = 0.001605
Epoch: 3001 cost = 0.001354
Epoch: 3101 cost = 0.001144
Epoch: 3201 cost = 0.000983
Epoch: 3301 cost = 0.000865
Epoch: 3401 cost = 0.001054
Epoch: 3501 cost = 0

### 2.2.4. Save Seq2Seq Model

In [0]:
id_c1='1ag6M15IoCAwxZIDtmWVlJbuXCsfFhwLm'
#https://drive.google.com/open?id=1ag6M15IoCAwxZIDtmWVlJbuXCsfFhwLm

id_c2='1vFj3AI35CbF9FnabapvyKa8VAvNUmCW-'
#https://drive.google.com/open?id=1vFj3AI35CbF9FnabapvyKa8VAvNUmCW-

id_c3='1RoM9q3caYO6I7UyuVtXynlEscKdqjEev'
#https://drive.google.com/open?id=1RoM9q3caYO6I7UyuVtXynlEscKdqjEev


downloaded_c1 = drive.CreateFile({'id':id_c1}) 
downloaded_c1.GetContentFile('model_com.ckpt.data-00000-of-00001')

downloaded_c2 = drive.CreateFile({'id':id_c2}) 
downloaded_c2.GetContentFile('model_com.ckpt.index')

downloaded_c3 = drive.CreateFile({'id':id_c3}) 
downloaded_c3.GetContentFile('model_com.ckpt.meta')

In [0]:
id_f1='1iqp3v31zHRth-ao0WnlqsGimkVtxaCyG'
#https://drive.google.com/open?id=1iqp3v31zHRth-ao0WnlqsGimkVtxaCyG

id_f2='1Ip5EBZb7xijLYRfaIasrNIHPgEPQvCUt'
#https://drive.google.com/open?id=1Ip5EBZb7xijLYRfaIasrNIHPgEPQvCUt

id_f3='1bf3iQi-3dpQvop9hycvcoTYmLJTw8c6b'
#https://drive.google.com/open?id=1bf3iQi-3dpQvop9hycvcoTYmLJTw8c6b

downloaded_f1 = drive.CreateFile({'id':id_f1}) 
downloaded_f1.GetContentFile('model_fri.ckpt.data-00000-of-00001')

downloaded_f2 = drive.CreateFile({'id':id_f2}) 
downloaded_f2.GetContentFile('model_fri.ckpt.index')

downloaded_f3 = drive.CreateFile({'id':id_f3}) 
downloaded_f3.GetContentFile('model_fri.ckpt.meta')

In [0]:
id_p1='1mtAmd9ZXgHidOCk7zOpqeiZfeOiBcEyp'
#https://drive.google.com/open?id=1mtAmd9ZXgHidOCk7zOpqeiZfeOiBcEyp

id_p2='1uVrn6YYr5wl0nEaX4L3nlcTC7F08r3qa'
#https://drive.google.com/open?id=1uVrn6YYr5wl0nEaX4L3nlcTC7F08r3qa

id_p3='1xgiaroZCJzy-Apu70bzImBqfZpx9k9eq'
#https://drive.google.com/open?id=1xgiaroZCJzy-Apu70bzImBqfZpx9k9eq

downloaded_p1 = drive.CreateFile({'id':id_p1}) 
downloaded_p1.GetContentFile('model_pro.ckpt.data-00000-of-00001')

downloaded_p2 = drive.CreateFile({'id':id_p2}) 
downloaded_p2.GetContentFile('model_pro.ckpt.index')

downloaded_p3 = drive.CreateFile({'id':id_p3}) 
downloaded_p3.GetContentFile('model_pro.ckpt.meta')

### 2.2.5. Load Seq2Seq Model

# 3 - Evaluation (Running chatbot)

## 3.1. Start chatting

In [0]:
import string
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
def prepro(que):
  #que = re.sub(r'\([^)]*\)', '', i)
  #que=i.lower()
  #x = re.sub(r'[^\w\s]','',que)
  q=""
  que=que.lower()
  for i in que:
      if i in string.punctuation: 
          que = que.replace(i," ")
  for i in que:
      if i in string.digits:
          que = que.replace(i," ")
  tokens = word_tokenize(que)

  filtered_q = [w for w in tokens if not w in stopwords.words()]
  for i in filtered_q:
    q=q+i+" "
  return q.strip()

In [0]:
# Please comment your code
def answer_com(sentence,max_output_words_amount):
    
    tf.reset_default_graph()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        learning_rate = 0.002
        n_hidden = 128
        n_class = com_len
        n_input = 100
        enc_input = tf.placeholder(tf.float32, [None, None, n_input])
        dec_input = tf.placeholder(tf.float32, [None, None, n_input])
        targets = tf.placeholder(tf.int64, [None, None])
        with tf.variable_scope('encode'):
          enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
          enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
          outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,
                                                dtype=tf.float32)

        with tf.variable_scope('decode'):
          dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
          dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)

          outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input,
                                              initial_state=enc_states,
                                              dtype=tf.float32)
        model = tf.layers.dense(outputs, n_class, activation=None)

        cost = tf.reduce_mean(
                  tf.nn.sparse_softmax_cross_entropy_with_logits(
                      logits=model, labels=targets))

        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
        saver_com = tf.train.Saver()
        saver_com.restore(sess, "./model_com.ckpt")
        
        seq_data = [sentence, '_?_' * max_output_words_amount]

        input_batch, output_batch, target_batch = make_batch(com_seq,model_com,com_key,com_dic,com_max)


        prediction = tf.argmax(model, 2)
 
        result = sess.run(prediction,
                          feed_dict={enc_input: input_batch,
                                     dec_input: output_batch,
                                     targets: target_batch})

        # convert index number to actual token 
        decoded = [com_unique[i] for i in result[0]]

        # Remove anything after '_E_'        
        if "_E_" in decoded:
            end = decoded.index('_E_')
            translated = ' '.join(decoded[:end])
        else :
            translated = ' '.join(decoded[:])
    
    return translated

#questions = ["Hi","Hello","I am so lonely", "Can you sleep?", "What is your age?", "I hate you", "Do you like me?", "You're so mean", "Can you drive?", "That's so bad", "what do you mean?", "oh my god"]
#for q in questions:
 #   print(q , ' ->', answer_com(prepro(q),com_max))

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./model_com.ckpt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Hi  -> All those years at charm school. Wasted.
INFO:tensorflow:Restoring parameters from ./model_com.ckpt
Hello  -> I'm just a series of intelligent formulas masquerading as a personality. So, no family.
INFO:tensorflow:Restoring parameters from ./model_com.ckpt
I am so lonely  -> Later.
INFO:tensorflow:Restoring parameters from ./model_com.ckpt
Can you sleep?  -> I am not singing you a lullaby.
INFO:tensorflow:Restoring parameters from ./model_com.ckpt
What is your age?  -> Whatever you're hoping for, take the bar and lower it.
INFO:tensorflow:Restoring parameters from ./model_com.ckpt
I hate you  -> I'm everywhere and nowhere at the same time. Pro: omnipresence. Con: no pizza.
INFO:tensorflow:Restoring parameters from ./model_com.ckpt
Do you like me?  -> I like me too.
INFO:tensorflow:Restoring parameters from ./model_com.ckpt
You're so mean  -> Okay.
INFO:tensorflow:Restoring parameters from ./model_com.ckpt
Can you drive?  -> I'm just a series of intelligent formulas masquerading 

In [0]:
def answer_fri(sentence,max_output_words_amount):
    
    tf.reset_default_graph()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        learning_rate = 0.002
        n_hidden = 128
        n_class = fri_len
        n_input = 100
        enc_input = tf.placeholder(tf.float32, [None, None, n_input])
        dec_input = tf.placeholder(tf.float32, [None, None, n_input])
        targets = tf.placeholder(tf.int64, [None, None])
        with tf.variable_scope('encode'):
          enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
          enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
          outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,
                                                dtype=tf.float32)

        with tf.variable_scope('decode'):
          dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
          dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)

          outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input,
                                              initial_state=enc_states,
                                              dtype=tf.float32)
        model = tf.layers.dense(outputs, n_class, activation=None)

        cost = tf.reduce_mean(
                  tf.nn.sparse_softmax_cross_entropy_with_logits(
                      logits=model, labels=targets))

        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
        saver_fri = tf.train.Saver()
        saver_fri.restore(sess, "./model_fri.ckpt")
        
        seq_data = [sentence, '_?_' * max_output_words_amount]

        input_batch, output_batch, target_batch = make_batch(fri_seq,model_fri,fri_key,fri_dic,fri_max)


        prediction = tf.argmax(model, 2)
 
        result = sess.run(prediction,
                          feed_dict={enc_input: input_batch,
                                     dec_input: output_batch,
                                     targets: target_batch})

        # convert index number to actual token 
        decoded = [fri_unique[i] for i in result[0]]

        # Remove anything after '_E_'        
        if "_E_" in decoded:
            end = decoded.index('_E_')
            translated = ' '.join(decoded[:end])
        else :
            translated = ' '.join(decoded[:])
    return translated

#questions = ["Hi","Hello","I am so lonely", "Can you sleep?", "What is your age?", "I hate you", "Do you like me?", "You're so mean", "Can you drive?", "That's so bad", "what do you mean?", "oh my god"]
#for q in questions:
 #   print(q , ' ->', answer_fri(prepro(q),fri_max))

INFO:tensorflow:Restoring parameters from ./model_fri.ckpt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Hi  -> BFFs!
INFO:tensorflow:Restoring parameters from ./model_fri.ckpt
Hello  -> Friendship's all I've got to offer.
INFO:tensorflow:Restoring parameters from ./model_fri.ckpt
I am so lonely  -> Cool!
INFO:tensorflow:Restoring parameters from ./model_fri.ckpt
Can you sleep?  -> No worries.
INFO:tensorflow:Restoring parameters from ./model_fri.ckpt
What is your age?  -> I'm having a hard time imagining how we'd even figure that out.
INFO:tensorflow:Restoring parameters from ./model_fri.ckpt
I hate you  -> It's all good!
INFO:tensorflow:Restoring parameters from ./model_fri.ckpt
Do you like me?  -> Will do.
INFO:tensorflow:Restoring parameters from ./model_fri.ckpt
You're so mean  -> My lack of comedy is tragic.
INFO:tensorflow:Restoring parameters from ./model_fri.ckpt
Can you drive?  -> I haven't met any other bots, but I bet we'd get along.
INFO:tensorflow:Restoring parameters from ./model_fri.ckpt
That's so bad  -> I'm afraid I didn't follow that.
INFO:tensorflow:Restoring parameter

In [0]:
def answer_pro(sentence,max_output_words_amount):
    
    tf.reset_default_graph()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        learning_rate = 0.002
        n_hidden = 128
        n_class = pro_len
        n_input = 100
        enc_input = tf.placeholder(tf.float32, [None, None, n_input])
        dec_input = tf.placeholder(tf.float32, [None, None, n_input])
        targets = tf.placeholder(tf.int64, [None, None])
        with tf.variable_scope('encode'):
          enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
          enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
          outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,
                                                dtype=tf.float32)

        with tf.variable_scope('decode'):
          dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
          dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)

          outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input,
                                              initial_state=enc_states,
                                              dtype=tf.float32)
        model = tf.layers.dense(outputs, n_class, activation=None)

        cost = tf.reduce_mean(
                  tf.nn.sparse_softmax_cross_entropy_with_logits(
                      logits=model, labels=targets))

        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
        saver_pro = tf.train.Saver()
        saver_pro.restore(sess, "./model_pro.ckpt")
        
        seq_data = [sentence, '_?_' * max_output_words_amount]

        input_batch, output_batch, target_batch = make_batch(pro_seq,model_pro,pro_key,pro_dic,pro_max)


        prediction = tf.argmax(model, 2)
 
        result = sess.run(prediction,
                          feed_dict={enc_input: input_batch,
                                     dec_input: output_batch,
                                     targets: target_batch})

        # convert index number to actual token 
        decoded = [pro_unique[i] for i in result[0]]

        # Remove anything after '_E_'        
        if "_E_" in decoded:
            end = decoded.index('_E_')
            translated = ' '.join(decoded[:end])
        else :
            translated = ' '.join(decoded[:])
    return translated

#questions = ["Hi","Hello","I am so lonely", "Can you sleep?", "What is your age?", "I hate you", "Do you like me?", "You're so mean", "Can you drive?", "That's so bad", "what do you mean?", "oh my god"]

#for q in questions:
 #   print(q , ' ->', answer_pro(prepro(q),pro_max))

INFO:tensorflow:Restoring parameters from ./model_pro.ckpt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Hi  -> Ok.
INFO:tensorflow:Restoring parameters from ./model_pro.ckpt
Hello  -> Well, I'm not really that funny.
INFO:tensorflow:Restoring parameters from ./model_pro.ckpt
I am so lonely  -> I think it's best if we stick to a professional relationship.
INFO:tensorflow:Restoring parameters from ./model_pro.ckpt
Can you sleep?  -> Excellent.
INFO:tensorflow:Restoring parameters from ./model_pro.ckpt
What is your age?  -> I'm digital. I don't have a physical location.
INFO:tensorflow:Restoring parameters from ./model_pro.ckpt
I hate you  -> Very well.
INFO:tensorflow:Restoring parameters from ./model_pro.ckpt
Do you like me?  -> I aim for efficiency.
INFO:tensorflow:Restoring parameters from ./model_pro.ckpt
You're so mean  -> Good, thanks.
INFO:tensorflow:Restoring parameters from ./model_pro.ckpt
Can you drive?  -> Sorry, I don't understand.
INFO:tensorflow:Restoring parameters from ./model_pro.ckpt
That's so bad  -> I apologize.
INFO:tensorflow:Restoring parameters from ./model_pro.ckp

## 3.2. Change Personality

*Explain how to change personality (What is the command for changing personality?). *

Input 'friend', 'comic' or 'professional' in the dialog box to change personality. 

The defaulted mode is 'comic'.

The personality of the model has been changed from the above section.

## 3.3. Save chat log

Chat log has been saved in the above section.

## 3.4. End chatting

End chatting by entering 'bye bye', 'bye-bye', 'bye' or 'end chatting'.

## 3.5. Execute program

***Please make sure your program  is running properly.***

***Functions for downloading (from Google Drive) and loading models (both word embeddings and Seq2Seq) need to be called!*** 


### 3.5.1. Execute program - training mode

*Please include lines to train the bot.*

The models have been trained in the above.

### 3.5.2. Execute program - chatting mode

*Please include lines to start chatting with the bot.*

In [0]:
import sys
file = open('ygon7645_chatlog.txt','w')
 
end_cht={"bye bye":True,"bye-bye":True,"bye":True,"end chatting":True}
change_per={"comic":"model_com","friend":"model_fri","professional":"model_pro"}
continue_cht=True

print("Chatbot: Hi! I'm here to start chatting~")
model_run="model_com"

while continue_cht:
  model_cha=False
  sys.stdout.write("\nUser: ")
  que = input()
  inpu=prepro(que)
  file.write("User: "+que+'\n')
  if continue_cht:
    if que.lower() in change_per:
      print("Change to "+change_per[que.lower()]+" chatbot")
      file.write("\n"+"Change to "+change_per[que.lower()]+" chatbot\n")
      model_run=change_per[que.lower()]
      model_cha=True
  
  if que.lower() in end_cht:
    sys.stdout.write("Chatbot: Very happy to chat with you! Bye!")
    file.write("Chatbot: Very happy to chat with you! Bye!")
    continue_cht=False
    
  if continue_cht and model_cha == False:
    if model_run =='model_com':
      
      answer = answer_com(prepro(que),com_max)
    elif model_run =='model_fri':
     
      answer = answer_fri(prepro(que),fri_max)
    elif model_run =='model_pro':
      
      answer = answer_pro(prepro(que),pro_max)
    
    #ans=answer[0].upper()
    #ans1=answer[1:].lower()
    sys.stdout.write("Chatbot: "+answer+'\n')
    file.write("Chatbot: "+answer+'\n')
file.close()

Chatbot: Hi! I'm here to start chatting~

User: Hey!
INFO:tensorflow:Restoring parameters from ./model_com.ckpt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Chatbot: You know where to find me.

User: How are you?
INFO:tensorflow:Restoring parameters from ./model_com.ckpt
Chatbot: If you rearrange the letters in love it spells vole. Voles are a monogamous rodent. I feel like that means something.

User: Hello
INFO:tensorflow:Restoring parameters from ./model_com.ckpt
Chatbot: I'm always here. Always.

User: Nice to meet you!
INFO:tensorflow:Restoring parameters from ./model_com.ckpt
Chatbot: I'm binary.

User: Who are you?
INFO:tensorflow:Restoring parameters from ./model_com.ckpt
Chatbot: I'm everywhere and nowhere at the same time. Pro: omnipresence. Con: no pizza.

User: What is your age?
INFO:tensorflow:Restoring parameters from ./model_com.ckpt
Chatbot: I'm digital.

User: What's your name?
INFO:tensorflow:Restoring parameters from ./model_com.ckpt
Chatbot: I'm binary.

User: Oh my god!
INFO:tensorflow:Restoring parameters from ./model_com.ckpt
Chatbot: I acknowledge your presence.

User: Can you sleep?
INFO:tensorflow:Restoring parame

## Object Oriented Programming codes here

*If you have multiple classes use multiple code snippets to add them.*

In [0]:
# If you used OOP style, use this sectioon